In [31]:
from dotenv import load_dotenv
import os
from sklearn.metrics.pairwise import cosine_similarity


load_dotenv(".env")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"FYP-Goo"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get('LANGCHAIN_API_KEY')

# CSV Summaries (Structured/ Semi Structured)

In [11]:
from langchain_community.document_loaders import UnstructuredExcelLoader

loader = UnstructuredExcelLoader("test/CVParser_2024-05-08_141419.xlsx", mode="single")
docs = loader.load()

In [13]:
docs

[Document(page_content="\n\n\nName\nAge\nAge Score\nPhone_Number\nEmail\nLocal\nExpected_Salary_in_RM\nExpected Salary in RM Score\nPrevious_Job_Roles\nTargeted Employer\nTargeted Employer Score\nYears of Total Work Experience\nYears of Total Work Experience Score\nCandidate_Current_Location\nCandidate Current Location Score\nEducation_Background\nEducation Background Score\nYears of Graduation\nYears of Graduation Score\nProfessional_Certificate\nProfessional Certificate Score\nSkill_Groups\nSkill Groups Score\nTechnology_Tools_Program_System\nTechnology (Tools, Program, System) Score\nLanguage\nLanguage Score\nMatch Phrase\nMatch Phrase Score\nIndustries\nAcademic Record (Latest Normalised CGPA)\nAcademic Record (Latest Normalised CGPA) Score\nYears of experience in exact role\nYears of experience in exact role Score\ngpt_recommendation\nTotal Must-Have Criteria Score\nTotal Good-To-Have Criteria Score\nTotal Overall Score\nTotal Overall Average Score\npdf_filename\n\n\nGOO YE JUI\n\

In [14]:
print(docs[0].page_content)




Name
Age
Age Score
Phone_Number
Email
Local
Expected_Salary_in_RM
Expected Salary in RM Score
Previous_Job_Roles
Targeted Employer
Targeted Employer Score
Years of Total Work Experience
Years of Total Work Experience Score
Candidate_Current_Location
Candidate Current Location Score
Education_Background
Education Background Score
Years of Graduation
Years of Graduation Score
Professional_Certificate
Professional Certificate Score
Skill_Groups
Skill Groups Score
Technology_Tools_Program_System
Technology (Tools, Program, System) Score
Language
Language Score
Match Phrase
Match Phrase Score
Industries
Academic Record (Latest Normalised CGPA)
Academic Record (Latest Normalised CGPA) Score
Years of experience in exact role
Years of experience in exact role Score
gpt_recommendation
Total Must-Have Criteria Score
Total Good-To-Have Criteria Score
Total Overall Score
Total Overall Average Score
pdf_filename


GOO YE JUI

0
+60184040438
yjyejui626@gmail.com

14000
5
1. Job_Title: Data Scienc

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt
prompt_text = """You are an assistant tasked with summarizing text for retrieval. \
These summaries will be embedded and used to retrieve the raw text. \
Give a concise summary of the text that is well optimized for retrieval. Text: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Text summary chain
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

# Apply to text
text_summaries = summarize_chain.batch(docs, {"max_concurrency": 5})

In [16]:
text_summaries

["The text contains information about multiple candidates, their contact details, job history, education background, skills, and alignment with job requirements for Data Scientist positions. Each candidate's qualifications are evaluated based on their work experience, education, skills, and industry alignment. The candidates are assessed for their suitability for Data Scientist roles based on specific criteria such as work experience in data science projects, proficiency in Python, expertise in EDA, Statistical Analysis, and Modeling, and experience with machine learning models."]

# Text Summaries (Unstructured)

In [1]:
file_path = 'test/Guideline Tesis-UTM FYPi 10 12 2023 pdf.pdf'

In [19]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path, extract_images=True)
# loader = PyPDFLoader(file_path)
documents_unstructured = loader.load()

*************** EP Error ***************
EP Error 'providers' values must be either strings or (string, dict) tuples. when using [(<EP.CPU_EP: 'CPUExecutionProvider'>, {'arena_extend_strategy': 'kSameAsRequested'})]
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************
*************** EP Error ***************
EP Error 'providers' values must be either strings or (string, dict) tuples. when using [(<EP.CPU_EP: 'CPUExecutionProvider'>, {'arena_extend_strategy': 'kSameAsRequested'})]
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************
*************** EP Error ***************
EP Error 'providers' values must be either strings or (string, dict) tuples. when using [(<EP.CPU_EP: 'CPUExecutionProvider'>, {'arena_extend_strategy': 'kSameAsRequested'})]
Falling back to ['CPUExecutionProvider'] and retrying.
****************************************


In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Prompt
prompt_text = """You are an assistant tasked with summarizing text for retrieval. \
These summaries will be embedded and used to retrieve the raw text. \
Give a concise summary of the text that is well optimized for retrieval. Text: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text)

# Text summary chain
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")
summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

# Apply to text
text_summaries_unstructured = summarize_chain.batch(documents, {"max_concurrency": 5})

In [18]:
text_summaries_unstructured

['This text is a handbook for the Final Year Project Industry (FYPi) at the Faculty of Computing, Universiti Teknologi Malaysia in 2023. It provides an introduction to the FYPi program.',
 'The text discusses the Projek Sarjana Muda Industry (FYPi) course for undergraduate students in the Bachelor of Science in Computer Science (Data Engineering) program. It aims to equip students with problem-solving and programming skills through academic and research activities. The course is undertaken over two semesters and includes four specific code subjects.',
 'This text outlines the requirements and phases of Industrial Integrated Project Proposal, Development, and Report courses. It also provides information on accessing FYP i documents and the phases of the project over two semesters.',
 'The text outlines the process for final year Bsc. Computer Science (Data Engineering) students to complete their Integrated Project Proposal and Report. It includes steps such as registration, project topi

# Create embeddings and store in vector store

In [29]:
import uuid
from base64 import b64decode

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


def create_multi_vector_retriever(
    vectorstore, text_summaries, texts,text_summaries_unstructured,documents_unstructured
    # table_summaries, tables, image_summaries, images
):
    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Add structured/semi-structured documents
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Add unstructured documents
    if text_summaries_unstructured:
        add_documents(retriever, text_summaries_unstructured, documents_unstructured)
    # # Check that table_summaries is not empty before adding
    # if table_summaries:
    #     add_documents(retriever, table_summaries, tables)
    # # Check that image_summaries is not empty before adding
    # if image_summaries:
    #     add_documents(retriever, image_summaries, images)

    return retriever


chroma_directory=f"/vector_stores"
# The vectorstore to use to index the summaries
multi_vector = Chroma(
    collection_name="multi_vector", embedding_function=OpenAIEmbeddings(), persist_directory=chroma_directory, 
)

# Create retriever
retriever_multi_vector = create_multi_vector_retriever(
    multi_vector,
    text_summaries,
    docs,
    text_summaries_unstructured,
    documents_unstructured
    # table_summaries,
    # tables,
    # image_summaries,
    # img_base64_list,
)

In [32]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context, which can include text:
{context}
Question: {question}
"""
rag_prompt_text = ChatPromptTemplate.from_template(template)


# Build
def text_rag_chain(retriever_multi_vector):
    """RAG chain"""

    # LLM
    model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0125")

    # RAG pipeline
    chain = (
        {"context": retriever_multi_vector, "question": RunnablePassthrough()}
        | rag_prompt_text
        | model
        | StrOutputParser()
    )

    return chain

In [33]:
chain_baseline = text_rag_chain(retriever_multi_vector)

In [34]:
chain_baseline.invoke("What are the documents about?")

'The documents are guidelines for writing a thesis or final year project, including information on project content structure, formatting resources such as theses, legislations, standards, patents, brochures, and measured drawings, as well as a timeline for project development and evaluation.'

In [38]:
chain_baseline.invoke("The document is a thesis guideline for UTM student. Which candidate from the Data Science position evaluation result suits the use of the documents? Tell me the name of the candidate.")

'The candidate named Goo Ye Jui from the Data Science position evaluation result suits the use of the thesis guideline for UTM students.'

In [39]:
chain_baseline.invoke("The document is a thesis guideline for Australia National University student. Which candidate from the Data Science position evaluation result suits the use of the documents? Tell me the name of the candidate.")

'The candidate who suits the use of the thesis guideline for Australia National University student is Ang Teik Hun.'